<a href="https://colab.research.google.com/github/Rice-from-data/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/Ned_Horsey_LS_DS3_143_Introduction_to_Bayesian_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 143

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.

### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

![Visualization of set intersection](https://upload.wikimedia.org/wikipedia/commons/9/99/Venn0001.svg)

Think of the overall rectangle as the whole probability space, $A$ as the left circle, $B$ as the right circle, and their intersection as the red area. Try to visualize the ratio being described in the above formula, and how it is different from just the $P(A)$ (not conditioned on $B$).

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.

### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

There's many examples of Bayes' theorem - one less absurd example is to apply to [breathalyzer tests](https://www.bayestheorem.net/breathalyzer-example/). You may think that a breathalyzer test that is 100% accurate for true positives (detecting somebody who is drunk) is pretty good, but what if it also has 8% false positives (indicating somebody is drunk when they're not)? And furthermore, the rate of drunk driving (and thus our prior belief)  is 1/1000.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 92% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drunk driving. Sounds like a job for Bayes' theorem!

$$
\begin{aligned}
P(Drunk | Positive) &= \frac{P(Positive | Drunk)P(Drunk)}{P(Positive)} \\
&= \frac{1 \times 0.001}{0.08} \\
&= 0.0125
\end{aligned}
$$

In other words, the likelihood that somebody is drunk given they tested positive with a breathalyzer in this situation is only 1.25% - probably much lower than you'd guess. This is why, in practice, it's important to have a repeated test to confirm (the probability of two false positives in a row is $0.08 * 0.08 = 0.0064$, much lower), and Bayes' theorem has been relevant in court cases where proper consideration of evidence was important.

##Derive Baye's Rule

\begin{align} 


$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

\end{align}

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

In [0]:
# # Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs

from scipy import stats
import numpy as np

# set a random seed
np.random.seed(seed=42)

# 1 coinflip, 50% prob, 100 flips
coinflips = np.random.binomial(n=1, p=0.5, size=100 )

print(coinflips)

[0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0
 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1
 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 0]


In [0]:
def confidence_interval(data, confidence=0.95):
  n = len(data)
  mean = sum(data)/n
  data = np.array(data)
  stderr = stats.sem(data)
  interval = stats.t.ppf((1+confidence) / 2.0, n-1) * stderr
  return(mean, mean-interval, mean+interval)

In [0]:
confidence_interval(coinflips)

(0.47, 0.3704689875017368, 0.5695310124982632)

In [0]:
stats.bayes_mvs(coinflips, 0.95)

(Mean(statistic=0.47, minmax=(0.37046898750173674, 0.5695310124982632)),
 Variance(statistic=0.25680412371134015, minmax=(0.1939698977025208, 0.3395533426586547)),
 Std_dev(statistic=0.5054540733507159, minmax=(0.44042013771229943, 0.5827120581030176)))

In [0]:
coinflips_mean_dist, _,_ = stats.mvsdist(coinflips)
coinflips_mean_dist

In [0]:
help(coinflips_mean_dist)

Help on rv_frozen in module scipy.stats._distn_infrastructure object:

class rv_frozen(builtins.object)
 |  # Frozen RV class
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dist, *args, **kwds)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  cdf(self, x)
 |  
 |  entropy(self)
 |  
 |  expect(self, func=None, lb=None, ub=None, conditional=False, **kwds)
 |  
 |  interval(self, alpha)
 |  
 |  isf(self, q)
 |  
 |  logcdf(self, x)
 |  
 |  logpdf(self, x)
 |  
 |  logpmf(self, k)
 |  
 |  logsf(self, x)
 |  
 |  mean(self)
 |  
 |  median(self)
 |  
 |  moment(self, n)
 |  
 |  pdf(self, x)
 |  
 |  pmf(self, k)
 |  
 |  ppf(self, q)
 |  
 |  rvs(self, size=None, random_state=None)
 |  
 |  sf(self, x)
 |  
 |  stats(self, moments='mv')
 |  
 |  std(self)
 |  
 |  var(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (

In [0]:
coinflips_mean_dist.interval(0.95)

(0.37046898750173674, 0.5695310124982632)

## Assignment - Code it up!

Most of the above was pure math - now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up, and as a stretch goal - refactor your code into helpful reusable functions!

Specific goals/targets:

1. Write a function `def prob_drunk_given_positive(prob_drunk_prior, prob_positive, prob_positive_drunk)` that reproduces the example from lecture, and use it to calculate and visualize a range of situations
2. Explore `scipy.stats.bayes_mvs` - read its documentation, and experiment with it on data you've tested in other ways earlier this week
3. Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach
4. In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/) - you could and should create something similar!

Stretch goals:

- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!

In [0]:
# TODO - code!

import matplotlib.pyplot as plt
from matplotlib_venn import venn3


def prob_drunk_given_positive(prob_drunk_prior = .001, prob_positive = 0.08, prob_positive_drunk = 1):
  
  '''
  Calculate Bayesian probability and display a population Venn diagram
  the first prob_drunk_prior is likelihood of drunk drivers in the population so 1/1000 
  is the default
  '''
  
  bayes_prob = (prob_positive_drunk * prob_drunk_prior) / prob_positive
  
  #Printed 
  answer1 = "The Bayesian probability that a person is drunk given that they tested positive on a breathalizer test is: "
  answer2 =  "\n And the probability of that same person taking a second brealizer test and having two false positives in a row is: "
  
  print(answer1 + str(bayes_prob) + answer2 + str(prob_positive**2))
  
  #venn diagram of the populations
#   v = venn3(subsets = (prob_drunk_prior, prob_positive, prob_positive_drunk, 0,0, 0,0), set_labels = ('Group A', 'Group B', 'Group C'))
#   v.get_label_by_id('A').set_text('Probability of any one driver being drunk')
# #   v.get_label_by_id('B').set_text('Bayesian probability a driver is drunk given a positive test')
#   v.get_label_by_id('C').set_text('false positive rate')
#   plt.show()
       
  

In [0]:
prob_drunk_given_positive()

The Bayesian probability that a person is drunk given that they tested positive on a breathalizer test is: 0.0125
 And the probability of that same person taking a second brealizer test and having two false positives in a row is: 0.0064


The above function illustrates the example of the breathalizer from class that has a 100% chance to detect positives but a 8% chance to false positive. If we change those values a little and add a chance for a false negative I wonder what happens?

In [0]:
# this function has the same parameters except now the test detects a positive only 92% of the time
prob_drunk_given_positive(.001, .08, .92)


The Bayesian probability that a person is drunk given that they tested positive on a breathalizer test is: 0.0115
 And the probability of that same person taking a second brealizer test and having two false positives in a row is: 0.0064


This is interesting, reducing the test accuracy by 8% reduced the probability of a true positive by only 1%.  This might be due to the 8% false positive rate on the denominator side being mirrored by the numerator.

Below I'll improt our congressional data and see if a bayesian analysis differs from our previous frequentest approach:

In [0]:
# Load in all data

df_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

names = ['Class Name', 
'handicapped-infants: 2 (y,n)', 
'water-project-cost-sharing: 2 (y,n)',
'adoption-of-the-budget-resolution: 2 (y,n)', 
'physician-fee-freeze: 2 (y,n)', 
'el-salvador-aid: 2 (y,n)',
'religious-groups-in-schools: 2 (y,n)', 
'anti-satellite-test-ban: 2 (y,n)', 
'aid-to-nicaraguan-contras: 2 (y,n)', 
'mx-missile: 2 (y,n)', 
'immigration: 2 (y,n)', 
'synfuels-corporation-cutback: 2 (y,n)', 
'education-spending: 2 (y,n)', 
'superfund-right-to-sue: 2 (y,n)', 
'crime: 2 (y,n)',
'duty-free-exports: 2 (y,n)', 
'export-administration-act-south-africa: 2 (y,n)']
# TODO - your code here!
import pandas as pd
import numpy as np
from scipy.stats  import ttest_ind

dfv = pd.read_csv(df_url, header=None, names=names)

dfv.columns = dfv.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(':_2_y,n', '')

boolean_filter = {'y' : 1, 'n' : 0, '?': np.nan}

dfv = dfv.replace(boolean_filter)

# dfv.head()

In [0]:
# I'll adapt my asses mean and CI function to also add Bayesian values

def assess_mean_and_CI_both(df, party, confidence=0.95):
  results=[]
  for col in df.select_dtypes(include=np.number):
    data = (df.loc[df['class_name']== party, col]).dropna()
    mean = (data).mean()
    n = len(data)
    stderr = stats.sem(data)
    
#     this is the freq interval calulation, the standard mean error * the percent point function
    interval = stderr * stats.t.ppf((1 + confidence) / 2., n-1)
    
#     bayesian values using mvsdist using the bayesian confidence intervals of bayes_mvs
    bayesian_mean_dist, _, _ =  stats.mvsdist(data)    
    b_mean =  bayesian_mean_dist.mean()
    b_lower, b_upper = bayesian_mean_dist.interval(confidence)[0], bayesian_mean_dist.interval(confidence)[1]
    
    results.append({'Issue': col,
                   'mean': mean,
                    'b_mean' : b_mean,
                    'lower CI' : mean - interval,
                    'bayesian lower CI' : b_lower,
                    'upper CI' : mean + interval,
                    'bayesian upper CI' : b_upper
                   })
  
  return pd.DataFrame(results)#.sort_values('mean')

In [0]:
democrats = assess_mean_and_CI_both(dfv, 'democrat')
democrats

,Issue,b_mean,bayesian lower CI,bayesian upper CI,lower CI,mean,upper CI
0,handicapped-infants,0.604651,0.544593,0.664710,0.544593,0.604651,0.664710
1,water-project-cost-sharing,0.502092,0.438245,0.565939,0.438245,0.502092,0.565939
2,adoption-of-the-budget-resolution,0.888462,0.849944,0.926980,0.849944,0.888462,0.926980
3,physician-fee-freeze,0.054054,0.026332,0.081776,0.026332,0.054054,0.081776
4,el-salvador-aid,0.215686,0.164863,0.266509,0.164863,0.215686,0.266509
5,religious-groups-in-schools,0.476744,0.415392,0.538097,0.415392,0.476744,0.538097
6,anti-satellite-test-ban,0.772201,0.720782,0.823620,0.720782,0.772201,0.823620
7,aid-to-nicaraguan-contras,0.828897,0.783085,0.874710,0.783085,0.828897,0.874710
8,mx-missile,0.758065,0.704394,0.811735,0.704394,0.758065,0.811735
9,immigration,0.471483,0.410757,0.532208,0.410757,0.471483,0.532208


They look absolutely identical so I will seperate the frequentist and bayesian functions and compare the two results.

In [0]:
def bayesian_mean_and_CI(df, party, confidence=0.95):
  results=[]
  for col in df.select_dtypes(include=np.number):
    data = (df.loc[df['class_name']== party, col]).dropna()
    mean = (data).mean()
    n = len(data)
    stderr = stats.sem(data)
    interval = stats.t.ppf((1+confidence) / 2.0, n-1) * stderr
    
    results.append({'Issue': col,
                   'mean': mean,
                    'lower CI' : mean - interval,
                    'upper CI' : mean + interval,                    
                   }) 
  return pd.DataFrame(results).sort_values('mean')

def assess_mean_and_CI(df, party, confidence=0.95):
  results=[]
  for col in df.select_dtypes(include=np.number):
    data = (df.loc[df['class_name']== party, col]).dropna()
    
#     bayesian values using mvsdist using the bayesian confidence intervals of bayes_mvs
    bayesian_mean_dist, _, _ =  stats.mvsdist(data)    
    b_mean =  bayesian_mean_dist.mean()
    b_lower, b_upper = bayesian_mean_dist.interval(confidence)[0], bayesian_mean_dist.interval(confidence)[1]
    
    results.append({'Issue': col,
                   'mean': b_mean,
                    'lower CI' : b_lower,
                    'upper CI' : b_upper
                   })
  return pd.DataFrame(results).sort_values('mean')

In [0]:
(bayesian_mean_and_CI(dfv, 'republican') == assess_mean_and_CI(dfv, 'republican')) 

,Issue,lower CI,mean,upper CI
14,True,True,True,True
8,True,True,True,True
10,True,False,True,True
2,True,True,True,True
7,True,False,True,False
0,True,True,True,True
6,True,True,True,True
1,True,True,True,True
9,True,True,True,True
15,True,True,True,True


In [0]:
(bayesian_mean_and_CI(dfv, 'democrat') == assess_mean_and_CI(dfv, 'democrat')) 

,Issue,lower CI,mean,upper CI
3,True,True,True,True
11,True,True,True,True
4,True,True,True,True
12,True,True,True,True
13,True,True,True,True
9,True,False,True,True
5,True,True,True,True
1,True,True,True,True
10,True,True,True,True
0,True,True,True,True


Comparing the truth tables accross parties it seems that the only differences between the Bayesian and frequentist mean and CI calculations are a few CI so I don't really see the point in graphing these differences.

In fact, based on the some reading I've done (https://jakevdp.github.io/blog/2014/06/12/frequentism-and-bayesianism-3-confidence-credibility/), it seems like the mathematical basis for a bayesian mean and CI are the same as the frequentist measures. 

The difference has to do with the interpretation of the data.

The Bayesian approach would be to say that given our data, we there is a  95% chance that the true mean lies within our  confidence interval.  Therefore the confidence interval reflects our beliefs about the data, while the true mean is random value within that interval.

Whereas the frequentist understanding would be that I am 95% confident that when I compute the CI from this data,  the population mean falls in this interval. Meaning that the population mean is a true value while our sample data is random

Honestly, this seems a bit philosophical but I would like to understand the practical differences (if any) between these two approaches.

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)